In [ ]:
import pandas as pd

df = pd.read_csv('population.csv')
print(df.head())

In [ ]:
df.head()

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("population.csv")

df = df[
    (df["IndicatorShortName"] == "Total population by sex") &
    (df["Sex"] == "Both sexes") &
    (df["Age"] == "Total") &
    (df["Time"].isin([2025, 2100]))
][["Location","Iso3","Time","Value"]]

df = (
    df.groupby(["Location","Iso3","Time"], as_index=False)["Value"]
      .first()
)

wide = df.pivot(index=["Location","Iso3"], columns="Time", values="Value").reset_index()
wide = wide.rename(columns={2025: "Pop_2025", 2100: "Pop_2100"})

wide["Change_Abs"] = wide["Pop_2100"] - wide["Pop_2025"]
wide["Change_Pct"] = np.where(
    wide["Pop_2025"] > 0,
    (wide["Change_Abs"] / wide["Pop_2025"]) * 100.0,
    np.nan
)

wide.rename(columns={"Location":"Country"}).to_csv("pop_change_2025_2100.csv", index=False)

print("Saved pop_change_2025_2100.csv with columns: Country, Iso3, Pop_2025, Pop_2100, Change_Abs, Change_Pct")


In [ ]:
import pandas as pd
import geopandas as gpd

world = gpd.read_file("countries/ne_110m_admin_0_countries.shp")
pop = pd.read_csv("pop_change_2025_2100.csv")

print(world.columns[:10])  # optional: just to confirm columns
print(pop.head())

merged = world.merge(pop, left_on="ADM0_A3", right_on="Iso3")

merged.to_file("pop_change_2025_2100.geojson", driver="GeoJSON")